<a href="https://colab.research.google.com/github/softmurata/promptparty/blob/main/datageneration/BackgroundDataAugumentationforAnswerImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!pip install -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q diffusers

In [ ]:
!pip install -q datasets

Fix Point

In [ ]:
# Please change this line in /usr/local/lib/python3.10/dist-packages/transformers/models/oneformer/image_processing_oneformer.py
# 1085, 1086
# after changing, please restart
"""
# class_queries_logits = outputs.class_queries_logits  # [batch_size, num_queries, num_classes+1]
# masks_queries_logits = outputs.masks_queries_logits  # [batch_size, num_queries, height, width]
class_queries_logits = outputs.transformer_decoder_class_predictions
masks_queries_logits = outputs.transformer_decoder_mask_predictions
"""

Demo data

In [ ]:
!wget https://img.freepik.com/free-photo/blank-living-room-interior-with-copy-space_43614-928.jpg -O /content/bg001.jpg
!wget https://img.freepik.com/free-photo/view-room-interior-with-furniture-copy-space_23-2150680550.jpg -O /content/bg002.jpg
!wget https://img.freepik.com/premium-photo/modern-interior-design-3d-illustration_259454-1911.jpg -O /content/bg003.jpg
!wget https://img.freepik.com/free-photo/green-plant-white-living-room-with-free-space_43614-871.jpg -O /content/bg004.jpg
!wget https://img.freepik.com/free-photo/japandi-living-room-interior-design_53876-145502.jpg -O /content/bg005.jpg

Main function

In [1]:
import torch
import os
from PIL import Image
import numpy as np
import cv2
from transformers import OneFormerProcessor, OneFormerModel
from torch import autocast
import json
import os
from diffusers import StableDiffusionImg2ImgPipeline

In [ ]:
# load oneformer model
model_id = "shi-labs/oneformer_ade20k_swin_large"
processor = OneFormerProcessor.from_pretrained(model_id)
model = OneFormerModel.from_pretrained(model_id)

# load stable diffusion model
model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
device = "cuda"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)

In [9]:
def calculate_bounding_box(mask_image):
    # Find the indices of non-zero pixels within the mask image
    non_zero_pixels = np.transpose(np.nonzero(mask_image))

    if non_zero_pixels.size == 0:
        # Return an empty Bounding Box if there are no non-zero pixels in the mask
        return None

    # Get x and y coordinates
    x_coords, y_coords = non_zero_pixels[:, 0], non_zero_pixels[:, 1]

    # Calculate the Bounding Box coordinates
    min_x, min_y = np.min(x_coords), np.min(y_coords)
    max_x, max_y = np.max(x_coords), np.max(y_coords)

    return (min_x, min_y, max_x, max_y)

def extract_selected_image(image_name, target_categories):
  img_path = f"/content/{image_name}.jpg"
  image = Image.open(img_path)

  inputs = processor(image, ["semantic"], return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs)

  # you can pass them to processor for semantic postprocessing
  predicted_semantic_map = processor.post_process_semantic_segmentation(
        outputs, target_sizes=[image.size[::-1]]
  )[0]

  item_list = model.config.label2id.keys()
  include_categories = {}
  include_categories["target"] = [model.config.label2id[item] for item in item_list if any(target in item for target in target_categories)]

  target_base_mask = np.zeros_like(predicted_semantic_map)
  source_image = cv2.imread(img_path)
  label_dict = {}
  for sub_id in include_categories["target"]:
    label_pred_map = np.where(predicted_semantic_map == sub_id, 255, 0)
    count = np.count_nonzero(label_pred_map == 255)
    if count > 0:
      bounding_box = calculate_bounding_box(label_pred_map)  # xmin, ymin, xmax, ymax
      ymin, xmin, ymax, xmax = bounding_box

      label = model.config.id2label[sub_id].split(",")[0].replace(" ", "")
      if label in label_dict.keys():
        label_dict[label] += 1
      else:
        label_dict[label] = 0

      target_image = source_image[ymin:ymax, xmin:xmax, :]

    target_base_mask += label_pred_map

  cv2.imwrite("/content/select.jpg", target_base_mask.astype(np.uint8))
  cv2.imwrite(f"/content/{image_name}_mask.jpg", target_base_mask.astype(np.uint8))
  # display(Image.fromarray(target_base_mask.astype(np.uint8)))

  # read the image
  img = cv2.imread("/content/select.jpg", 0)

  # binarize the image
  binr = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

  # define the kernel
  kernel = np.ones((4, 4), np.uint8)

  # invert the image
  invert = cv2.bitwise_not(binr)

  # erode the image
  erosion = cv2.erode(invert, kernel,
                      iterations=1)

  erosion_invert = cv2.bitwise_not(erosion)
  cv2.imwrite(f"/content/{image_name}_mask.jpg", erosion_invert)

  display(Image.fromarray(erosion_invert))


def create_alphablend(label_name, bg_name, ratio, x, y):
  fg_img = cv2.imread(f"/content/{label_name}.jpg")
  fh, fw = fg_img.shape[:2]
  bg_img = cv2.imread(f"/content/{bg_name}.jpg")
  bh, bw = bg_img.shape[:2]
  bg_img = cv2.resize(bg_img, (int(bw * ratio), int(bh * ratio)))

  mask = cv2.cvtColor(cv2.imread(f"/content/{label_name}_mask.jpg"), cv2.COLOR_BGR2GRAY)
  mask = cv2.resize(mask, (fw, fh))

  w = min(fg_img.shape[1], bg_img.shape[1] - x)
  h = min(fg_img.shape[0], bg_img.shape[0] - y)

  fg_roi = fg_img[:h, :w]
  bg_roi = bg_img[y:y+h, x:x +w]

  bg_roi[:] = np.where(mask[:h, :w, np.newaxis] == 0, bg_roi, fg_roi)

  rgb = cv2.resize(cv2.cvtColor(bg_img, cv2.COLOR_BGR2RGB), (bw, bh))
  display(Image.fromarray(rgb))
  cv2.imwrite(f"/content/{label_name}_blend.jpg", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR))



def refine_with_sd(image_name, bg_name, save_dir, strength):
  prompt = ""
  init_img = Image.open(f"/content/{image_name}_blend.jpg")

  # パイプラインの実行
  generator = torch.Generator(device).manual_seed(42) # 再現できるようにseedを設定
  with torch.autocast("cuda"):
      image = pipe(prompt, image=init_img, guidance_scale=7.5, strength=strength, generator=generator).images[0]

  image.save(f"{save_dir}/{image_name}_{bg_name}.jpg")
  display(image)


Preprocess function

In [ ]:
image_name = "room"
target_categories = ["sofa", "table", "cushion", "rug", "chair", "computer"]

extract_selected_image(image_name, target_categories)

In [ ]:
ratio = 2
x, y = 100, 100
bg_name = "bg005"
create_alphablend(image_name, bg_name, ratio, x, y)

In [ ]:
save_dir = "/content/dreamboothimages"
os.makedirs(save_dir, exist_ok=True)
refine_with_sd(image_name, bg_name, save_dir, 0.3)

Finetuning

In [27]:
%cd /content
!git clone https://github.com/huggingface/diffusers.git

/content
Cloning into 'diffusers'...
remote: Enumerating objects: 40712, done.
remote: Counting objects: 100% (521/521), done.
remote: Compressing objects: 100% (280/280), done.
remote: Total 40712 (delta 309), reused 362 (delta 194), pack-reused 40191
Receiving objects: 100% (40712/40712), 27.22 MiB | 19.52 MiB/s, done.
Resolving deltas: 100% (30140/30140), done.


In [ ]:
import os
os.makedirs("/content/normal/room", exist_ok=True)

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
%cd /content/diffusers/examples/research_projects/multi_subject_dreambooth
import json

# here we are using parameters for prior-preservation and validation as well.
concepts_list = [
    {
        "instance_prompt":      "photo of a <new1> room",
        "class_prompt":         "photo of a room",
        "instance_data_dir":    "/content/dreamboothimages/",
        "class_data_dir":       "/content/normal/room",
    },
]

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

Training

In [ ]:
%cd /content/diffusers/examples/research_projects/multi_subject_dreambooth
!accelerate launch train_multi_subject_dreambooth.py \
  --pretrained_model_name_or_path="SG161222/Realistic_Vision_V5.1_noVAE"  \
  --output_dir="testsinglecheckpoints" \
  --train_text_encoder \
  --with_prior_preservation \
  --prior_loss_weight=1.0 \
  --concepts_list="concepts_list.json" \
  --num_class_images=200 \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1500 \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --use_8bit_adam \
  --set_grads_to_none

Inference

In [6]:
%cd /content/diffusers/examples/research_projects/multi_subject_dreambooth
from diffusers import StableDiffusionPipeline
import torch

model_id = "testsinglecheckpoints"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

/content/diffusers/examples/research_projects/multi_subject_dreambooth


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
prompt = "a <new1> room, best quality, realistic, super detailed"
negative_prompt = "worst quality, ugly, deformed, tiling, poor quality, picture frame, poorly drawn, mutation, duplicate"
generator = torch.Generator(device="cuda").manual_seed(-1)
images = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=100,
    guidance_scale=7.5,
    num_images_per_prompt=4
).images


for img in images:
  display(img)